In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor

In [ ]:
def remove_label(df, label):
    x = df.drop(label, axis=1)
    y = df[label].copy()
    return (x,y)

In [ ]:
train_data = pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv", index_col=0)
train_data

In [ ]:
train_data.isna().any()

In [ ]:
train_data.info()

In [ ]:
train_set, val_set = train_test_split(train_data, test_size=0.3, random_state=42, shuffle=True, stratify=None)
val_set, test_set = train_test_split(val_set, test_size=0.5, random_state=42, shuffle=True, stratify=None)

In [ ]:
x_train, y_train = remove_label(train_set, "FloodProbability")
x_val, y_val = remove_label(val_set, "FloodProbability")
x_test, y_test = remove_label(test_set, "FloodProbability")

In [ ]:
cat = CatBoostRegressor(iterations=1500, learning_rate=0.07, min_data_in_leaf=70, depth=15, l2_leaf_reg=5, custom_metric=['R2'], eval_metric='R2', task_type='GPU')
cat.fit(x_train, y=y_train, eval_set=(x_val, y_val), plot=True, use_best_model=True)

In [ ]:
y_pred = cat.predict(x_test)
np.round(y_pred, 3)

In [ ]:
y_test

In [ ]:
print("R2: {0}".format(np.sqrt(r2_score(y_test, y_pred))))

In [ ]:
test_data = pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv", index_col=0)
test_data

In [ ]:
submission = cat.predict(test_data)
submission

In [ ]:
submission_df = pd.DataFrame(submission, index=test_data.index, columns=['FloodProbability'])
submission_df

In [ ]:
submission_df.to_csv("/kaggle/working/submission.csv")